In [1]:
import yaml
import pandas as pd
from sqlalchemy import create_engine

In [2]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    configSource = config['source']
    configDestination = config['destination']

urlSource = f"{configSource['driver']}://{configSource['user']}:{configSource['password']}@{configSource['host']}:{configSource['port']}/{configSource['db']}"
urlDestination = f"{configDestination['driver']}://{configDestination['user']}:{configDestination['password']}@{configDestination['host']}:{configDestination['port']}/{configDestination['db']}"

engineSource = create_engine(urlSource)
engineDestination = create_engine(urlDestination)

In [5]:
client = pd.read_sql_table('cliente', engineSource)
client.drop(columns=["nombre_contacto", "ciudad_id", "tipo_cliente_id", "activo", "coordinador_id", "sector", "nit_cliente"], inplace=True)
client.rename(columns={
  "cliente_id": "client_id", 
  "nombre": "name", 
  "direccion": "address",
  "telefono": "phone"
  }, inplace=True)
client["saved"] = pd.to_datetime("now")

client

,client_id,name,email,address,phone,saved
0,1,Cliente 2,algo.com,Calle 100 No 25-18,327-00000,2024-10-23 13:58:04.853288
1,2,Cliente 1,algo.com,Calle 100 No 25-18,327-00000,2024-10-23 13:58:04.853288
2,6,CLINICA DEPORTIVA DEL SUR,algo.com,Calle 100 No 25-18,327-00000,2024-10-23 13:58:04.853288
3,19,HOSPITAL ORTOPEDICO DE COLOMBIA,algo.com,Calle 100 No 25-18,327-00000,2024-10-23 13:58:04.853288
4,8,CLINICA NEFROLOGOS DE CALI,algo.com,Calle 100 No 25-18,327-00000,2024-10-23 13:58:04.853288
5,3,BANCO REGIONAL DE SANGRE BLOD-LIFE,algo.com,Calle 100 No 25-18,327-00000,2024-10-23 13:58:04.853288
6,4,CRUZ AZUL-LIFE,algo.com,Calle 100 No 25-18,327-00000,2024-10-23 13:58:04.853288
7,5,CLINICA CALI -JOVEN,algo.com,Calle 100 No 25-18,327-00000,2024-10-23 13:58:04.853288
8,7,CLINICA COFFE -HEALTH,algo.com,Calle 100 No 25-18,327-00000,2024-10-23 13:58:04.853288
9,9,UNIDAD DE TRAUMA DEL OESTE,algo.com,Calle 100 No 25-18,327-00000,2024-10-23 13:58:04.853288


In [6]:
client.to_sql("dim_client", engineDestination, index=False, if_exists="replace")

27